In [5]:
# load the data

# rownames are both X if importing from R output, "Model" / "Dataset" if importing from spreadsheet
mdl_res <- read.csv("25-04_us-floods_model-res.csv", row.names = "Model")
obs_res <- read.csv("25-04_us-floods_obs-res.csv", row.names = "Dataset")

# Table of evaluation results

In [6]:
# list variables that have evaluation results
eval_cnm <- unique(sapply(strsplit(colnames(mdl_res)[grepl("eval", colnames(mdl_res))], "_"), "[", 2))

In [12]:
# loop over those variables, process those columns into a string
eval_res <- do.call("cbind", sapply(eval_cnm, function(ec) {
    obs_eval <- obs_res[,grepl(ec, colnames(obs_res))]
    mdl_eval <- mdl_res[,grepl(ec, colnames(mdl_res))]
    colnames(mdl_eval) <- gsub("eval_", "", colnames(mdl_eval))
    
    df <- round(rbind(obs_eval, mdl_eval), 3)
    
    apply(df, 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")"))
    
}, simplify = F))

    
# then just need to merge with whatever other columns are in the model results (specify that all rows should be kept)
eval_res <- merge(eval_res, mdl_res[,c("sc", "sp", "conclusion", "include"),drop = F], by = 0, all = T)

# tidy up the table (reorder by obs then models)
rownames(eval_res) <- eval_res$Row.names
eval_res <- eval_res[c(rownames(obs_res), rownames(mdl_res)),-1]
    

In [14]:
write.csv(eval_res, "table-eval_us-floods.csv")

# Table of model fitting results

In [109]:
attr_cnm <- c("PR", "dI")

In [110]:
mdl_keep = mdl_res[mdl_res$include == "y",]

In [114]:
attr_res <- do.call("cbind", sapply(attr_cnm, function(ec) {
    obs_eval <- obs_res[,grepl(ec, colnames(obs_res))]
    mdl_eval <- mdl_keep[,grepl(paste0("attr_",ec), colnames(mdl_keep))]
    colnames(mdl_eval) <- gsub("attr_", "", colnames(mdl_eval))
    
    df <- signif(rbind(obs_eval, mdl_eval), 3)
    
    apply(df, 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")"))
    
}, simplify = F))
    
proj_res <- do.call("cbind", sapply(attr_cnm, function(ec) {
    mdl_eval <- mdl_keep[,grepl(paste0("proj_",ec), colnames(mdl_keep))]
    
    df <- signif(mdl_eval, 3)
    
    apply(df, 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")"))
    
}, simplify = F))
colnames(proj_res) <- paste0("proj_", colnames(proj_res))

# then just need to merge the two
all_res <- merge(attr_res, proj_res, by = 0, all = T, sort = F)
rownames(all_res) <- all_res$Row.names
all_res <- all_res[,-1]

# and also add the event magnitude
ev_magnitude <- round(rbind(setNames(obs_res[,"ev",drop = F], c("rl10")), mdl_keep[,"rl10",drop = F]), 0)
all_res <- merge(ev_magnitude, all_res, by = 0, all = T, sort = F)

# # reorder by obs then models
# all_res <- all_res[c(rownames(obs_res), rownames(mdl_keep)),]

write.csv(all_res, "table-results_us-floods.csv")    
all_res

Row.names,rl10,PR,dI,proj_PR,proj_dI
<I<chr>>,<dbl>,<chr>,<chr>,<chr>,<chr>
CPC,179,"2.03 (0.242, 12.2)","13.6 (-10.1, 46.8)",NA,NA
ERA5,146,"4.96 (1.07, 1.74e+10)","26.3 (1.27, 69.7)",NA,NA
MSWEP,208,"2.44 (0.45, 664)","22.5 (-12.5, 79.6)",NA,NA
ClimGrid,159,"2.24 (0.343, 16)","16 (-13.1, 50.1)",NA,NA
CAM-MPAS-HR,133,"0.895 (0.115, 7.27)","-1.59 (-24.6, 28.2)","NA (NA, NA)","NA (NA, NA)"
CAM-MPAS-LR,114,"2.93 (0.133, 63500)","9.29 (-18.5, 45.3)","NA (NA, NA)","NA (NA, NA)"
CMCC-CM2-HR4,89,"0.684 (0.125, 12.7)","-3.7 (-21.9, 20.8)","NA (NA, NA)","NA (NA, NA)"
CMCC-CM2-VHR4,128,"159 (3.2, Inf)","35.7 (13.8, 62)","NA (NA, NA)","NA (NA, NA)"
HadGEM3-GC31-HM,123,"15.6 (0.214, Inf)","14.7 (-13.8, 41.6)","NA (NA, NA)","NA (NA, NA)"
